In [2]:
import requests
import folium
from shapely.geometry import shape  # Optional for advanced geometry; not strictly needed here

# Step 1: Fetch Active Weather Alerts from NWS API
NWS_ALERTS_URL = "https://api.weather.gov/alerts/active"
response = requests.get(NWS_ALERTS_URL, headers={"User-Agent": "weather-demo"})
alerts = response.json()

# Step 2: Parse Alert Features and Handle Polygon / MultiPolygon
alert_data = []

for feature in alerts.get("features", []):
    props = feature["properties"]
    geom = feature.get("geometry")
    
    if not geom:
        continue
    
    geom_type = geom["type"]
    event = props.get("event", "Unknown Event")
    severity = props.get("severity", "Unknown")
    area_desc = props.get("areaDesc", "No area description")

    # Prepare a list of polygons for both Polygon and MultiPolygon
    if geom_type == "Polygon":
        # Single polygon (wrap it to treat uniformly)
        polygons = [geom["coordinates"]]
    elif geom_type == "MultiPolygon":
        # MultiPolygon is a list of polygons
        polygons = geom["coordinates"]
    else:
        # If it's some other geometry (LineString, Point, etc.), skip it
        continue

    # Each entry in `polygons` is one polygon
    for coords in polygons:
        alert_data.append({
            "event": event,
            "severity": severity,
            "area": area_desc,
            "coordinates": coords
        })

# Step 3: Create a Folium Map Centered on the US
us_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Step 4: Define Severity Colors
severity_colors = {
    "Extreme": "red",
    "Severe": "orange",
    "Moderate": "yellow",
    "Minor": "blue",
    "Unknown": "gray"
}

# Step 5: Add Polygons to the Map
for alert in alert_data:
    severity = alert["severity"]
    color = severity_colors.get(severity, "gray")

    # NWS coordinates come in [longitude, latitude] order
    # Folium expects [latitude, longitude]
    # Here we handle only the "outer ring" (the first element in a polygon's coordinates array).
    # If you need the full ring set (including holes), you'd iterate over each sub-ring.
    coords = alert["coordinates"][0]  # outer ring
    coords_swapped = [(lat, lon) for lon, lat in coords]

    folium.Polygon(
        locations=coords_swapped,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.4,
        tooltip=f"{alert['event']} ({alert['severity']})",
        popup=(
            f"<b>Event:</b> {alert['event']}<br>"
            f"<b>Severity:</b> {alert['severity']}<br>"
            f"<b>Area:</b> {alert['area']}"
        )
    ).add_to(us_map)

# Step 6: Display or Save the Map
# In a Jupyter notebook, just show the map by making us_map the last line:
us_map

# If running as a standalone Python script, you can save the map to an HTML file:
# us_map.save("nws_alerts_map.html")
